In [1]:
import json
import os
from dataclasses import dataclass
from openai import OpenAI
import yaml
import pandas as pd
import textwrap

# Ensure no limitations in Jupyter Notebook output
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

client = OpenAI()

In [2]:
long_list_table = [
  {
    "#": "1",
    "Risk Driver": "Loan-to-Value Ratio (LTV)",
    "Missing Value Rate": "2%",
    "Data Availability": "99%",
    "p-value": "0.001",
    "Gini": "0.28",
    "Annual Stability": "Stable",
    "IV": "0.23",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "2",
    "Risk Driver": "Debt-to-Income Ratio (DTI)",
    "Missing Value Rate": "5%",
    "Data Availability": "98%",
    "p-value": "0.010",
    "Gini": "0.22",
    "Annual Stability": "Stable",
    "IV": "0.15",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "3",
    "Risk Driver": "Credit Score",
    "Missing Value Rate": "0%",
    "Data Availability": "100%",
    "p-value": "0.0005",
    "Gini": "0.35",
    "Annual Stability": "Stable",
    "IV": "0.32",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "4",
    "Risk Driver": "Borrower Age",
    "Missing Value Rate": "3%",
    "Data Availability": "97%",
    "p-value": "0.045",
    "Gini": "0.07",
    "Annual Stability": "Stable",
    "IV": "0.05",
    "WoE Trend": "Mostly smooth",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "5",
    "Risk Driver": "Property Type",
    "Missing Value Rate": "1%",
    "Data Availability": "95%",
    "p-value": "0.001",
    "Gini": "0.10",
    "Annual Stability": "Stable",
    "IV": "0.08",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "6",
    "Risk Driver": "Region",
    "Missing Value Rate": "0%",
    "Data Availability": "90%",
    "p-value": "0.090",
    "Gini": "0.05",
    "Annual Stability": "Stable",
    "IV": "0.03",
    "WoE Trend": "Slightly wavy",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep (borderline)"
  },
  {
    "#": "7",
    "Risk Driver": "Months in Current Job",
    "Missing Value Rate": "15%",
    "Data Availability": "93%",
    "p-value": "0.030",
    "Gini": "0.12",
    "Annual Stability": "Stable",
    "IV": "0.10",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "8",
    "Risk Driver": "Payment History Score",
    "Missing Value Rate": "2%",
    "Data Availability": "99%",
    "p-value": "0.0002",
    "Gini": "0.30",
    "Annual Stability": "Stable",
    "IV": "0.20",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "9",
    "Risk Driver": "Monthly Payment",
    "Missing Value Rate": "0%",
    "Data Availability": "92%",
    "p-value": "0.002",
    "Gini": "0.25",
    "Annual Stability": "Stable",
    "IV": "0.18",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "10",
    "Risk Driver": "Loan Term (Years)",
    "Missing Value Rate": "1%",
    "Data Availability": "100%",
    "p-value": "0.040",
    "Gini": "0.06",
    "Annual Stability": "Stable",
    "IV": "0.04",
    "WoE Trend": "Mostly smooth",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "11",
    "Risk Driver": "Income Verification Flag",
    "Missing Value Rate": "3%",
    "Data Availability": "95%",
    "p-value": "0.006",
    "Gini": "0.20",
    "Annual Stability": "Stable",
    "IV": "0.12",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "12",
    "Risk Driver": "Number of Credit Lines",
    "Missing Value Rate": "25%",
    "Data Availability": "90%",
    "p-value": "0.070",
    "Gini": "0.04",
    "Annual Stability": "Flips",
    "IV": "0.03",
    "WoE Trend": "Erratic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Drop (Stability)"
  },
  {
    "#": "13",
    "Risk Driver": "Employment Type (Full/Part)",
    "Missing Value Rate": "8%",
    "Data Availability": "89%",
    "p-value": "0.150",
    "Gini": "0.03",
    "Annual Stability": "Stable",
    "IV": "0.02",
    "WoE Trend": "Slightly wavy",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Drop (p-value)"
  },
  {
    "#": "14",
    "Risk Driver": "Co-Borrower Indicator",
    "Missing Value Rate": "12%",
    "Data Availability": "88%",
    "p-value": "0.025",
    "Gini": "0.09",
    "Annual Stability": "Stable",
    "IV": "0.07",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "15",
    "Risk Driver": "Homeownership Tenure (Years)",
    "Missing Value Rate": "18%",
    "Data Availability": "92%",
    "p-value": "0.015",
    "Gini": "0.11",
    "Annual Stability": "Stable",
    "IV": "0.12",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "16",
    "Risk Driver": "Previous Mortgage Experience",
    "Missing Value Rate": "2%",
    "Data Availability": "97%",
    "p-value": "0.002",
    "Gini": "0.27",
    "Annual Stability": "Stable",
    "IV": "0.25",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "17",
    "Risk Driver": "Number of Dependents",
    "Missing Value Rate": "10%",
    "Data Availability": "90%",
    "p-value": "0.045",
    "Gini": "0.08",
    "Annual Stability": "Stable",
    "IV": "0.06",
    "WoE Trend": "Mostly smooth",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "18",
    "Risk Driver": "Interest-Only Payment Flag",
    "Missing Value Rate": "25%",
    "Data Availability": "82%",
    "p-value": "0.080",
    "Gini": "0.05",
    "Annual Stability": "Stable",
    "IV": "0.03",
    "WoE Trend": "Slightly wavy",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep (borderline)"
  },
  {
    "#": "19",
    "Risk Driver": "Mortgage Insurance Indicator",
    "Missing Value Rate": "0%",
    "Data Availability": "99%",
    "p-value": "0.004",
    "Gini": "0.18",
    "Annual Stability": "Stable",
    "IV": "0.14",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "20",
    "Risk Driver": "Self-Employed Flag",
    "Missing Value Rate": "1%",
    "Data Availability": "87%",
    "p-value": "0.060",
    "Gini": "0.03",
    "Annual Stability": "Stable",
    "IV": "0.03",
    "WoE Trend": "Slightly wavy",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep (borderline)"
  },
  {
    "#": "21",
    "Risk Driver": "Purpose (Refinance/Purchase)",
    "Missing Value Rate": "2%",
    "Data Availability": "99%",
    "p-value": "0.010",
    "Gini": "0.13",
    "Annual Stability": "Stable",
    "IV": "0.10",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "22",
    "Risk Driver": "Outstanding Revolving Credit",
    "Missing Value Rate": "15%",
    "Data Availability": "85%",
    "p-value": "0.020",
    "Gini": "0.16",
    "Annual Stability": "Stable",
    "IV": "0.14",
    "WoE Trend": "Mostly smooth",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "23",
    "Risk Driver": "Original Loan Amount",
    "Missing Value Rate": "0%",
    "Data Availability": "100%",
    "p-value": "0.0005",
    "Gini": "0.24",
    "Annual Stability": "Stable",
    "IV": "0.18",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "24",
    "Risk Driver": "Seasoning of Loan (Months on Book)",
    "Missing Value Rate": "5%",
    "Data Availability": "95%",
    "p-value": "0.005",
    "Gini": "0.14",
    "Annual Stability": "Stable",
    "IV": "0.11",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "25",
    "Risk Driver": "FICO Score at Origination",
    "Missing Value Rate": "0%",
    "Data Availability": "100%",
    "p-value": "0.0001",
    "Gini": "0.33",
    "Annual Stability": "Stable",
    "IV": "0.27",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "26",
    "Risk Driver": "Bankruptcy History (Y/N)",
    "Missing Value Rate": "10%",
    "Data Availability": "90%",
    "p-value": "0.012",
    "Gini": "0.20",
    "Annual Stability": "Stable",
    "IV": "0.17",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "27",
    "Risk Driver": "Late Payment Count (Last 12 mo)",
    "Missing Value Rate": "3%",
    "Data Availability": "95%",
    "p-value": "0.002",
    "Gini": "0.29",
    "Annual Stability": "Stable",
    "IV": "0.21",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "28",
    "Risk Driver": "Cash Reserves at Closing",
    "Missing Value Rate": "8%",
    "Data Availability": "88%",
    "p-value": "0.022",
    "Gini": "0.06",
    "Annual Stability": "Stable",
    "IV": "0.04",
    "WoE Trend": "Mostly smooth",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "29",
    "Risk Driver": "Property Valuation Variance (Appraisal)",
    "Missing Value Rate": "22%",
    "Data Availability": "78%",
    "p-value": "0.035",
    "Gini": "0.08",
    "Annual Stability": "Flips",
    "IV": "0.05",
    "WoE Trend": "Erratic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Drop (Data Avail & Stability)"
  },
  {
    "#": "30",
    "Risk Driver": "Rate Type (Fixed/Variable)",
    "Missing Value Rate": "0%",
    "Data Availability": "100%",
    "p-value": "0.005",
    "Gini": "0.15",
    "Annual Stability": "Stable",
    "IV": "0.11",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "31",
    "Risk Driver": "Borrower Marital Status",
    "Missing Value Rate": "6%",
    "Data Availability": "93%",
    "p-value": "0.010",
    "Gini": "0.04",
    "Annual Stability": "Stable",
    "IV": "0.03",
    "WoE Trend": "Slightly wavy",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "32",
    "Risk Driver": "Delinquency in Last 24 mo (Count)",
    "Missing Value Rate": "5%",
    "Data Availability": "92%",
    "p-value": "0.002",
    "Gini": "0.20",
    "Annual Stability": "Stable",
    "IV": "0.15",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "33",
    "Risk Driver": "Overdraft Fee Incidence",
    "Missing Value Rate": "35%",
    "Data Availability": "62%",
    "p-value": "0.040",
    "Gini": "0.07",
    "Annual Stability": "Stable",
    "IV": "0.06",
    "WoE Trend": "Mostly smooth",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Drop (Data Avail <80%)"
  },
  {
    "#": "34",
    "Risk Driver": "Cross-Sell Product Ownership",
    "Missing Value Rate": "40%",
    "Data Availability": "58%",
    "p-value": "0.100",
    "Gini": "0.02",
    "Annual Stability": "Stable",
    "IV": "0.01",
    "WoE Trend": "Slightly wavy",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Drop (Missing + p-value + IV)"
  },
  {
    "#": "35",
    "Risk Driver": "Remaining Term (Months)",
    "Missing Value Rate": "3%",
    "Data Availability": "92%",
    "p-value": "0.030",
    "Gini": "0.11",
    "Annual Stability": "Stable",
    "IV": "0.09",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "36",
    "Risk Driver": "Number of Open Trade Lines",
    "Missing Value Rate": "15%",
    "Data Availability": "85%",
    "p-value": "0.055",
    "Gini": "0.09",
    "Annual Stability": "Stable",
    "IV": "0.06",
    "WoE Trend": "Slightly wavy",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "37",
    "Risk Driver": "Type of Employment Contract",
    "Missing Value Rate": "12%",
    "Data Availability": "88%",
    "p-value": "0.110",
    "Gini": "0.04",
    "Annual Stability": "Stable",
    "IV": "0.03",
    "WoE Trend": "Mostly smooth",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Drop (p-value)"
  },
  {
    "#": "38",
    "Risk Driver": "Historical Balance Growth (Avg %)",
    "Missing Value Rate": "3%",
    "Data Availability": "90%",
    "p-value": "0.002",
    "Gini": "0.12",
    "Annual Stability": "Stable",
    "IV": "0.08",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "39",
    "Risk Driver": "Credit Utilization Ratio",
    "Missing Value Rate": "5%",
    "Data Availability": "92%",
    "p-value": "0.0008",
    "Gini": "0.25",
    "Annual Stability": "Stable",
    "IV": "0.18",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "40",
    "Risk Driver": "Length of Credit History (Years)",
    "Missing Value Rate": "2%",
    "Data Availability": "96%",
    "p-value": "0.012",
    "Gini": "0.16",
    "Annual Stability": "Stable",
    "IV": "0.10",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "41",
    "Risk Driver": "Recent Credit Inquiries (Last 6 mo)",
    "Missing Value Rate": "10%",
    "Data Availability": "85%",
    "p-value": "0.008",
    "Gini": "0.14",
    "Annual Stability": "Stable",
    "IV": "0.09",
    "WoE Trend": "Mostly smooth",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "42",
    "Risk Driver": "Cash-Out Refinancing Indicator",
    "Missing Value Rate": "20%",
    "Data Availability": "80%",
    "p-value": "0.050",
    "Gini": "0.05",
    "Annual Stability": "Stable",
    "IV": "0.03",
    "WoE Trend": "Slightly wavy",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep (borderline)"
  },
  {
    "#": "43",
    "Risk Driver": "Occupancy Status (Primary/Other)",
    "Missing Value Rate": "1%",
    "Data Availability": "99%",
    "p-value": "0.005",
    "Gini": "0.09",
    "Annual Stability": "Stable",
    "IV": "0.07",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "44",
    "Risk Driver": "CLTV (Combined Loan-to-Value)",
    "Missing Value Rate": "7%",
    "Data Availability": "93%",
    "p-value": "0.001",
    "Gini": "0.21",
    "Annual Stability": "Stable",
    "IV": "0.17",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "45",
    "Risk Driver": "Housing Expense Ratio",
    "Missing Value Rate": "5%",
    "Data Availability": "95%",
    "p-value": "0.003",
    "Gini": "0.23",
    "Annual Stability": "Stable",
    "IV": "0.18",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "46",
    "Risk Driver": "Property Location (Urban/Suburban/Rural)",
    "Missing Value Rate": "6%",
    "Data Availability": "92%",
    "p-value": "0.070",
    "Gini": "0.08",
    "Annual Stability": "Stable",
    "IV": "0.06",
    "WoE Trend": "Mostly smooth",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep (borderline)"
  },
  {
    "#": "47",
    "Risk Driver": "Residual Income (After Mortgage)",
    "Missing Value Rate": "5%",
    "Data Availability": "95%",
    "p-value": "0.002",
    "Gini": "0.26",
    "Annual Stability": "Stable",
    "IV": "0.22",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "48",
    "Risk Driver": "Payment Shock (Change in Payment)",
    "Missing Value Rate": "12%",
    "Data Availability": "90%",
    "p-value": "0.005",
    "Gini": "0.18",
    "Annual Stability": "Stable",
    "IV": "0.14",
    "WoE Trend": "Monotonic",
    "Business/Reg.": "Complies",
    "Practical": "Yes",
    "Decision": "Keep"
  },
  {
    "#": "49",
    "Risk Driver": "Balloon Payment Flag",
    "Missing Value Rate": "1%",
    "Data Availability": "99%",
    "p-value": "0.030",
    "Gini": "0.08",
    "Annual Stability": "Flips",
    "IV": "0.05",
    "WoE Trend": "Erratic",
    "Business/Reg.": "Violates",
    "Practical": "Yes",
    "Decision": "Drop (Regulatory)"
  },
  {
    "#": "50",
    "Risk Driver": "Relationship Length w/ Bank (Years)",
    "Missing Value Rate": "0%",
    "Data Availability": "100%",
    "p-value": "0.025",
    "Gini": "0.10",
    "Annual Stability": "Stable",
    "IV": "0.07",
    "WoE Trend": "Mostly smooth",
    "Business/Reg.": "Complies",
    "Practical": "No",
    "Decision": "Drop (Practical)"
  }
]


In [3]:
df = pd.DataFrame(long_list_table)
df.head()

,#,Risk Driver,Missing Value Rate,Data Availability,p-value,Gini,Annual Stability,IV,WoE Trend,Business/Reg.,Practical,Decision
0,1,Loan-to-Value Ratio (LTV),2%,99%,0.001,0.28,Stable,0.23,Monotonic,Complies,Yes,Keep
1,2,Debt-to-Income Ratio (DTI),5%,98%,0.010,0.22,Stable,0.15,Monotonic,Complies,Yes,Keep
2,3,Credit Score,0%,100%,0.0005,0.35,Stable,0.32,Monotonic,Complies,Yes,Keep
3,4,Borrower Age,3%,97%,0.045,0.07,Stable,0.05,Mostly smooth,Complies,Yes,Keep
4,5,Property Type,1%,95%,0.001,0.10,Stable,0.08,Monotonic,Complies,Yes,Keep


In [4]:
PROMPT_TEXT = """
Below is an example **prompt** you can give to your LLM. It explains each characteristic in your table, provides thresholds for acceptability, describes their relative importance, and instructs the model on how to combine individual assessments into a final decision. You can adapt the wording and details based on your specific use case.

---

**System / Instruction**:
You are a risk analytics assistant. You will receive risk drivers, each with specific characteristics. For each risk driver, **evaluate** the characteristics in detail (with **Gini** as the primary criterion) and **combine** them into an overall assessment, then produce:

1. A **decision**: one of "keep", "drop", or "analyze by expert".
2. A **justification**: a short explanation (1–2 sentences) describing why you made that decision, **explicitly highlighting the Gini first**.
3. A **rating**: an integer from 1 to 10 indicating overall quality.

---

**Characteristic Descriptions & Thresholds**:
1. **Missing Value Rate**
   - **Definition**: Percentage of missing or null entries for this driver.
   - **Importance**: High. Too many missing values reduces reliability.
   - **Acceptability Threshold**: <5% is ideal, 5–10% is borderline, >10% is concerning.

2. **Data Availability**
   - **Definition**: How easily or consistently this driver is available (e.g., 0.97 means data for 97% of records).
   - **Importance**: High. If availability is <90%, the driver may not be useful.
   - **Acceptability Threshold**: >95% is good, 90–95% is marginal, <90% is problematic.

3. **p-value**
   - **Definition**: Indicates statistical significance when measuring correlation or effect.
   - **Importance**: Medium. A very low p-value (<0.05) suggests the driver is statistically relevant.
   - **Acceptability Threshold**: <0.05 is typically good, 0.05–0.10 is borderline, >0.10 is weak significance.

4. **Gini**
   - **Definition**: Measures discriminatory power of the driver (0 = random, 1 = perfect discrimination).
   - **Importance**: **Primary**. This should be the first criterion considered.
   - **Acceptability Threshold**: >0.2 is somewhat useful, >0.3 is strong, <0.1 is poor.

5. **Annual Stability**
   - **Definition**: How stable this driver’s predictive relationship is year over year (e.g., “Stable” or “Volatile”).
   - **Importance**: High. Unstable drivers can degrade over time.
   - **Acceptability Threshold**: Must be “Stable” or close to stable for long-term use.

6. **IV** (Information Value)
   - **Definition**: Indicates predictive power (similar to Gini, but different scale).
   - **Importance**: Medium to high.
   - **Acceptability Threshold**: >0.1 is moderate, >0.3 is strong, <0.02 is poor.

7. **WoE Trend** (Weight of Evidence trend)
   - **Definition**: Shows whether binning or grouping is logically consistent (monotonic or smooth is usually ideal).
   - **Importance**: Medium.
   - **Acceptability Threshold**: “Monotonic” or “Mostly smooth” is good. Inconsistent or erratic signals are red flags.

8. **Business/Reg.**
   - **Definition**: Indicates if the driver is acceptable from a business and regulatory standpoint.
   - **Importance**: High. Even a strong driver can’t be used if it violates regulations.
   - **Acceptability Threshold**: Must comply with internal policy or external regulation.

9. **Practical**
   - **Definition**: Whether the driver is easy to interpret and implement.
   - **Importance**: Medium. A highly complex driver can be less practical.
   - **Acceptability Threshold**: Typically “Yes” means feasible, “No” means cost or complexity issues.

---

**Combining Individual Assessments**:
- Evaluate **Gini** first. If Gini is too low (<0.1), strongly consider dropping or further expert analysis.
- Then assess **Missing Value Rate**, **Data Availability**, **p-value**, and **Annual Stability**.
- Check **IV** and **WoE Trend** for consistent predictive performance.
- Ensure there are no red flags for **Business/Reg.** or **Practical** usage.
- If multiple red flags are present, consider dropping or sending for expert analysis.
- If the driver meets thresholds (especially Gini), recommend keeping it.

---
Example Justifications (with Gini first):

"The Gini is 0.32, indicating strong predictive power; even though the missing rate is 7%, this driver’s overall performance remains robust enough to keep."
"The Gini is only 0.08, which is too low to be informative; despite decent stability and low missing values, it’s best to drop this driver."

---

**Single Driver Input Structure**:
You will receive data for one driver at a time as a JSON object in the following format (values are examples):

[single_driver_assessment] = {single_driver_assessment}

"""



In [5]:
def shortlist_evaluator(row):
    single_driver_assessment = row.to_json()
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": PROMPT_TEXT.format(single_driver_assessment=single_driver_assessment),
            }
        ],
        temperature=0,
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "rate",
                    "description": "Assess individual risk drivers according to provided criteria. Return the value of the risk driver on a 10-point scale and the justification behind this assessment.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "driver_value": {"type": "integer", "minimum": 1, "maximum": 10},
                            "justification": {"type": "string"},
                        },
                        "required": ["correctness_rating", "clarity_rating", "justification"],
                    },
                },
            }
        ],
        tool_choice={"type": "function", "function": {"name": "rate"}},
    )
    arguments = json.loads(response.choices[0].message.tool_calls[0].function.arguments)

    print(arguments)
    return (arguments["driver_value"], arguments["justification"])

In [6]:
# Apply the function to each row and create two new columns
dfa = df.head().copy()

dfa[['driver_value', 'justification']] = dfa.apply(
    lambda row: shortlist_evaluator(row), 
    axis=1, 
    result_type='expand'
)

dfa

{'justification': 'The Gini is 0.28, which indicates somewhat useful predictive power. The missing value rate is 2%, which is ideal, and data availability is 99%, which is good. The p-value is 0.001, indicating strong statistical significance. The annual stability is stable, which is favorable. The IV is 0.23, showing moderate predictive power, and the WoE trend is monotonic, which is good. The driver complies with business and regulatory requirements and is practical to implement. Overall, the driver meets the necessary thresholds, especially with a Gini above 0.2, so it is recommended to keep it.', 'driver_value': 8}
{'justification': 'The Gini is 0.22, indicating somewhat useful predictive power. The missing value rate is 5%, which is borderline but acceptable. Data availability is high at 98%, and the p-value of 0.010 suggests strong statistical significance. The driver is stable annually, with a moderate IV of 0.15 and a monotonic WoE trend. It complies with business and regulator

,#,Risk Driver,Missing Value Rate,Data Availability,p-value,Gini,Annual Stability,IV,WoE Trend,Business/Reg.,Practical,Decision,driver_value,justification
0,1,Loan-to-Value Ratio (LTV),2%,99%,0.001,0.28,Stable,0.23,Monotonic,Complies,Yes,Keep,8,"The Gini is 0.28, which indicates somewhat useful predictive power. The missing value rate is 2%, which is ideal, and data availability is 99%, which is good. The p-value is 0.001, indicating strong statistical significance. The annual stability is stable, which is favorable. The IV is 0.23, showing moderate predictive power, and the WoE trend is monotonic, which is good. The driver complies with business and regulatory requirements and is practical to implement. Overall, the driver meets the necessary thresholds, especially with a Gini above 0.2, so it is recommended to keep it."
1,2,Debt-to-Income Ratio (DTI),5%,98%,0.010,0.22,Stable,0.15,Monotonic,Complies,Yes,Keep,7,"The Gini is 0.22, indicating somewhat useful predictive power. The missing value rate is 5%, which is borderline but acceptable. Data availability is high at 98%, and the p-value of 0.010 suggests strong statistical significance. The driver is stable annually, with a moderate IV of 0.15 and a monotonic WoE trend. It complies with business and regulatory requirements and is practical to implement. Overall, the driver meets the necessary thresholds for retention."
2,3,Credit Score,0%,100%,0.0005,0.35,Stable,0.32,Monotonic,Complies,Yes,Keep,9,"The Gini is 0.35, indicating strong predictive power. The missing value rate is 0%, and data availability is 100%, both of which are excellent. The p-value is 0.0005, showing strong statistical significance. Annual stability is stable, and the IV is 0.32, which is strong. The WoE trend is monotonic, and it complies with business and regulatory requirements. It is also practical to implement. Overall, this driver is robust and should be kept."
3,4,Borrower Age,3%,97%,0.045,0.07,Stable,0.05,Mostly smooth,Complies,Yes,Keep,3,"The Gini is 0.07, which is below the acceptable threshold of 0.1, indicating poor discriminatory power. Despite having a stable annual stability and a low missing value rate of 3%, the low Gini suggests that this driver is not informative enough to be kept without further expert analysis."
4,5,Property Type,1%,95%,0.001,0.10,Stable,0.08,Monotonic,Complies,Yes,Keep,5,"The Gini is 0.10, which is on the borderline of being useful. The missing value rate is low at 1%, and data availability is acceptable at 95%. The p-value is very low at 0.001, indicating strong statistical significance. The driver is stable annually, has a moderate IV of 0.08, and a monotonic WoE trend. It complies with business and regulatory requirements and is practical to implement. Overall, the driver has some strengths but the Gini is a concern."


In [7]:
def create_summary(data):
    summary_data = data[['Risk Driver', 'driver_value', 'justification']].to_json()
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {   
                "role": "developer", 
                "content": "You are a credit risk expert and need to write a half a summary of the outcomes in latex. \
                            Do not use tables of other structured formats. \
                            Return response as valid LATEX code assuming that this file will be integrated with main.tex"},
            {
                "role": "user",
                "content": f"Table: {summary_data}"
            }
        ]
    )
    
    return completion.choices[0].message


kala = create_summary(dfa)

In [8]:
# Split text into lines
lines = kala.content.split("\n")

# Build a list of lines to keep
filtered_lines = []
for line in lines:
    # Skip lines starting with ```latex, ```, or \section
    if line.startswith("```latex"):
        continue
    if line.startswith("```"):
        continue
    if line.startswith("\\section"):
        continue
    filtered_lines.append(line)

# Rejoin the lines you want to keep
kala = "\n".join(filtered_lines)

print(kala)


In evaluating the efficacy of various risk drivers, it's imperative to assess their predictive power using the Gini coefficient, statistical significance, and data reliability, among other factors.

For the \textbf{Loan-to-Value Ratio (LTV)}, we observed a Gini of 0.28, suggesting useful predictive power. The data is robust with only 2\% missing values and 99\% availability. With a p-value of 0.001, the statistical significance is notable. This driver is stable annually and complies with both business and regulatory standards, making it a strong candidate for retention.

The \textbf{Debt-to-Income Ratio (DTI)} showcased a Gini of 0.22. Despite having a higher missing value rate of 5\%, its statistical significance (p-value = 0.010) and high data availability (98\%) support its stability. Meeting necessary compliance and practical implementation criteria, it also meets the threshold for retention.

With regard to \textbf{Credit Score}, its Gini of 0.35 demonstrates strong predictive po

In [9]:
# Save the LaTeX to a file
with open("doc_generation/sections/Purpose.tex", "w") as file:
    file.write(kala)

print("LaTeX document saved to summary.tex")

LaTeX document saved to summary.tex
